### CSCIE104 HW2 Problem 4 

Create a Dask cluster and run a few queries on some Wikimedia traffic data, using Dask dataframe.


__1. Create a Client__ and request 4 workers, 1 thread, and 750MB of RAM each.

In [71]:
from dask.distributed import Client
import dask, dask.dataframe as dd

import distributed
import sys
import pandas as pd

cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, memory_limit='750MB') 
client = distributed.Client(cluster)
client

/Users/mpellegrin008/anaconda3/envs/pytorch/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60909 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60909/status,
Dashboard: http://127.0.0.1:60909/status,Workers: 4
Total threads: 4,Total memory: 2.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60910,Workers: 4
Dashboard: http://127.0.0.1:60909/status,Total threads: 4
Started: Just now,Total memory: 2.79 GiB
Comm: tcp://127.0.0.1:60923,Total threads: 1
Dashboard: http://127.0.0.1:60926/status,Memory: 715.26 MiB
Nanny: tcp://127.0.0.1:60913,


__2. Read data__ from the `pageviews_small.csv` file. Use Dask's `blocksize=` parameter to set each partition to max of 10 MB.
 

In [72]:
import dask.dataframe

import dask.dataframe

ddf = dask.dataframe.read_csv('/Users/mpellegrin008/Documents/VS Code/CSCI104_Advanced_DL/Week 2/Problem4-Dask-Notebook-Data/pageviews_small.csv', blocksize=1e7, delimiter = ' ')

In [73]:
ddf

,en.m,Article_51,1,0
npartitions=3,,,,
,object,object,int64,int64
,...,...,...,...
,...,...,...,...
,...,...,...,...


__3. Change the column names__ to `project`, `page`, `requests`, and `x` then drop the `x` column.

In [74]:
ddf.columns = ['project', 'page', 'requests', 'x']

In [75]:
ddf = ddf.drop(columns=['x'])

In [76]:
ddf.map_partitions(type).compute()

/Users/mpellegrin008/anaconda3/envs/pytorch/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/Users/mpellegrin008/anaconda3/envs/pytorch/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/Users/mpellegrin008/anaconda3/envs/pytorch/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further d

0    <class 'pandas.core.frame.DataFrame'>
1    <class 'pandas.core.frame.DataFrame'>
2    <class 'pandas.core.frame.DataFrame'>
dtype: object

__4. Filter__ for `project` matching "en" (English Wikipedia)

In [77]:
en = ddf[ddf.project.str.contains('en',na=False)].head()
en

/Users/mpellegrin008/anaconda3/envs/pytorch/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


,project,page,requests
2,en,Panorama_(La_Dispute_album),1
8,en.m,File:AvalonBeach.jpg,1
9,en,Operating_cost,4
10,en.m,Grumman_Albatross,2
14,en.m,Priyanka_Purohit,12


__5. Count__ how many pages were accessed from English Wikipedia vs. all projects in this dataset. (Note: each project/page combination appears on a unique line, so this amounts to just counting records)

In [78]:
ddf = ddf.assign(requests=ddf['requests'].astype(pd.Int64Dtype()))


In [79]:
# ... (count all)

all = ddf['page'].count().compute()
all

1118987

In [80]:
# ... (count English)

eng_count = en['requests'].count()

In [81]:
print(eng_count)

5


In [82]:
# Dont forget to close the client when you're done...

client.close()